<a href="https://colab.research.google.com/github/kangnurrohman/unstructured-data/blob/main/src/text%20data/Part%204%20-%20Document%20Similarity%20%26%20Keyword%20Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/data/data 5.zip", 'r')
zip_ref.extractall("data")
zip_ref.close()

In [3]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')

sw_indo = stopwords.words("indonesian") + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Import Data

In [4]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


## Extract TFIDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  % sorted(inconsistent)


## TFIDF Similarity -> Document Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [9]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]])

In [10]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [11]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [12]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [13]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

## Keyword Extraction

In [14]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [15]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219])

In [17]:
vocab[169219]

'ginandjar'

In [18]:
vocab[436652]

'putusan'

In [19]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [20]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names_out()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [21]:
teks = """Jakarta - Rektor Universitas Sultan Agung (Unissula) Semarang Prof Dr Gunarto, SH. MHum. mendapatkan penghargaan pada PWI Jateng Award 2022 yang dilaksanakan di Gradika Bakti Praja Kota Semarang, Jumat (25/3). Penyerahan dilakukan oleh Ketua PWI Jawa Tengah Amir Machmud NS.

Gunarto mendapatkan penghargaan atas gagasan kreatif pengembangan budaya akademik 'kampus birrul walidain'. Itu merupakan sebuah gagasan tentang pendidikan karakter unggul untuk mendorong para mahasiswa pada khususnya serta dosen dan karyawan pada umumnya, untuk senantiasa meningkatkan kebaktian pada ibu dan ayah masing masing.

Gunarto menekankan mahasiswa bukan hanya dididik untuk menjadi manusia unggul dan kompeten dalam pekerjaan, namun juga memiliki kesadaran dan tanggung jawab moral untuk berbakti kepada kedua orang tua. Ia memandang nilai-nilai seperti itu semakin terkikis belakangan ini seiring dengan perkembangan dunia yang sangat drastic. Oleh sebab itu universitas perlu mengingatkan pentingnya etika terhadap orang tua tersebut.

Penghargaan PWI Jateng Award 2022 menjadi salah satu rangkaian dari perayaan Hari Pers Nasional (HPN). Ajang penganugerahan ini merupakan apresiasi atas prestasi, kontribusi, dan rekam jejak para tokoh dan korporasi dalam pembangunan di Jawa Tengah melalui profesi dan peran tanggung jawab sosial.

Selain Rektor Unissula, terdapat 15 tokoh dan 3 korporasi yang mendapatkan penghargaan dalam acara PWI Jateng award 2022, antara lain Gubernur Jateng Ganjar Pranowo, politisi senior Partai Golkar asal Pati Firman Soebagyo, dan politisi PDIP Dewi Aryani, Bupati Brebes Idza Priyanti Mukti Bupati Pemalang Agung Wibowo, Walikota Tegal Dedy Yon Supriyono, Wakil Wali Kota Semarang Hevearita G. Rahayu, dan Bupati Magelang Zainal Arifin.

Penghargaan juga diberikan pada Rektor USM Dr Ir Supari Priambodo, MT, Rektor Universitas Pancasakti Tegal Dr Taufiqulloh, MHum, Ketua DPRD Kabupaten Tegal Moch Faiq, Dandim 0733/ Kota Semarang Letkol Inf Honi Havana. Direktur PT Industri Jamu dan Farmasi Sido Muncul, Tbk. Irwan Hidayat, Direktur Utama Bank Jateng Supriyatno, serta Dirut PT Nisajana Hasna Rizqy Khurotul Jannah juga mendapatkan penghargaan di ajang tersebut.

Sementara itu, tiga korporasi yang dianugerahi penghargaan, yakni PT Semen Gresik (Persero), PT Pertamina Patra Niaga Regional Jawa Bagian Tengah, dan ExxonMobil Cepu Limited (EMCL). (Content Promotion/Unissula)

(ega/ega)

content promotion"
"""

In [22]:
extract_keywords_tfidf(teks, tfidf)

['penghargaan',
 'pwi',
 'gunarto',
 'jateng',
 'award',
 'kota semarang',
 'korporasi',
 'rektor',
 'tegal',
 'semarang']